In [1]:
import dask.dataframe as dd
from tqdm import tqdm
import pandas as pd
import numpy as np

In [ ]:
user_ids = data.user_id.unique()
course_weeks = data.course_week.unique()

for i, user_id in tqdm(user_ids.iteritems()):
    print(user_id)
    user_rows = data.loc[(data['user_id'] == user_id), :]

    active_course_weeks = user_rows.loc[:, 'course_week'].unique()

    zero_course_weeks = course_weeks[~course_weeks.isin(active_course_weeks.compute())]

    started_week = user_rows.loc[:, 'user_started_week'].min().compute()
    last_active_week = user_rows.loc[:, 'user_last_active_week'].min().compute()

    completed_week = user_rows.loc[:, 'user_completed_week'].min().compute()

#     zero_course_weeks = zero_course_weeks.between(left=0, right=int(last_active_week))
    
    zero_course_weeks = zero_course_weeks.loc[zero_course_weeks < int(last_active_week)]
    

    zero_course_weeks = zero_course_weeks.loc[zero_course_weeks > 0]

    print(zero_course_weeks.compute())
    
    

    zero_rows_df = pd.DataFrame(
        np.zeros((zero_course_weeks.count().compute(), len(data.columns))),
        columns=data.columns
    )

    zero_rows_df.loc[:, 'user_id'] = user_id
    zero_rows_df.loc[:, 'course_week'] = zero_course_weeks
    zero_rows_df.loc[:, 'user_started_week'] = started_week
    zero_rows_df.loc[:, 'user_last_active_week'] = last_active_week
    zero_rows_df.loc[:, 'user_completed_week'] = completed_week

    
    data = dd.concat([data, zero_rows_df], axis=0, interleave_partitions=True)
#     raise ValueError('check')
    print(data)
#     data = data.append(zero_rows_df)

In [126]:
len(data.compute())

182112

In [1]:
import pandas as pd

In [16]:
data = pd.read_csv('data/Microsoft+DAT206x+3T2017/model_data_l.csv')
data['user_dropped_out_next_week'].value_counts()

0    46571
1    19500
Name: user_dropped_out_next_week, dtype: int64

In [17]:
d = pd.read_csv('data/Microsoft+DAT247x+4T2017/model_data.csv')
d_l = pd.read_csv('data/Microsoft+DAT247x+4T2017/model_data_l.csv')

In [21]:
d['user_dropped_out_next_week'].value_counts()

0    2533
1     544
Name: user_dropped_out_next_week, dtype: int64

In [22]:
d_l['user_dropped_out_next_week'].value_counts()

0    617
1    544
Name: user_dropped_out_next_week, dtype: int64

In [6]:
from sklearn.metrics import accuracy_score

In [7]:
accuracy_score(d['user_dropped_out_next_week'], d['predicted_user_dropped_out_next_week'])

0.89470263243418913

In [8]:
y_true = d['user_dropped_out_next_week']

In [9]:
y_true.shape

(3077,)

In [10]:
import numpy as np
np.zeros(y_true.shape)

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [11]:
accuracy_score(d['user_dropped_out_next_week'], np.zeros(y_true.shape))

0.82320441988950277

In [2]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, confusion_matrix

In [384]:
model_accs = []
base_accs = []
precs = []
recs = []
f1s = []
t1s = []
cks = []
course_ids = []


with open('{}/top_course_ids.txt'.format('data')) as top_course_ids:
    for top_course_id in top_course_ids:
        if '4T2017' in top_course_id and 'JPN' not in top_course_id and 'DAT102x' not in top_course_id:
            try:
                final = pd.read_csv('data/{}/model_data_with_preds_l.csv'.format(top_course_id.strip()))
                course_ids.append(top_course_id.strip())
                y_true = final['user_dropped_out_next_week']
                y_pred = final['predicted_user_dropped_out_next_week']
            except:
                print('Course: {} did not have predictions.'.format(top_course_id))
                continue
                
                
            base_acc = np.zeros(y_true.shape)


            model_accs.append(accuracy_score(y_true, y_pred))
            base_accs.append(accuracy_score(y_true, base_acc))


            precs.append(precision_score(y_true, y_pred))
            recs.append(recall_score(y_true, y_pred))
            
            f1s.append(f1_score(y_true, y_pred))

            conf_matrix = confusion_matrix(y_true, y_pred)
            t1s.append(conf_matrix[1][0] / len(y_true))

            cks.append(cohen_kappa_score(y_true, y_pred))
                
            
            

Course: Microsoft+DAT201x+4T2017
 did not have predictions.
Course: Microsoft+DAT215.4x+4T2017
 did not have predictions.


In [385]:
scores = pd.DataFrame(
    [],
    columns=[
        'course_id', 'model_accuracy', 'base_accuracy (all 0)',
        'precision', 'recall', 'f1_score', 'num_false_negatives',
        'cohens_kappa'
    ]
)

In [386]:
scores['course_id'] = course_ids
scores['model_accuracy'] = model_accs
scores['base_accuracy (all 0)'] = base_accs
# scores['precision'] = precs
scores['recall'] = recs
scores['f1_score'] = f1s
scores['false_neg_perc'] = t1s
scores['cohens_kappa'] = cks

In [387]:
scores.describe()

,model_accuracy,base_accuracy (all 0),recall,f1_score,cohens_kappa,false_neg_perc
count,34.000000,34.000000,34.000000,34.000000,34.000000,34.000000
mean,0.684881,0.433035,0.891028,0.757141,0.307221,0.061064
std,0.039352,0.092054,0.047550,0.052653,0.053306,0.029501
min,0.615760,0.244774,0.744044,0.665376,0.183833,0.019920
25%,0.648871,0.346731,0.868089,0.719372,0.278210,0.040122
50%,0.684636,0.461572,0.896361,0.748100,0.314175,0.056655
75%,0.716498,0.495224,0.928215,0.809384,0.338812,0.073257
max,0.769920,0.580146,0.971510,0.855172,0.408120,0.166392


In [3]:
# Test
models = []

In [4]:
from cntk.device import try_set_default_device, gpu
try_set_default_device(gpu(0))

True

In [35]:
from collections import Counter
import numpy as np
import pandas as pd

from cntk.device import try_set_default_device, gpu
try_set_default_device(gpu(0))

import keras
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Sequential, load_model, Model
from keras.layers import Activation, Dense, Dropout, Input, Average
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras.utils import plot_model
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# from sklearn.cross_validation import train_test_split
from pipeline.util import *


def get_data_path():
    return './data'

def save_df_to_file(dataframe, name, course_id, type='csv'):
    """
    Save a dataframe to a csv file in the data/{course_id}/ directory
    """

    path = '{}/{}/{}'.format(get_data_path(), course_id, name)

    if type == 'excel':
        path += '.xlsx'
        create_directory_safe(path)
        dataframe.to_excel(path)
    else:
        path += '.csv'
        create_directory_safe(path)
        dataframe.to_csv(path, index=False)
        
        
def get_data(current_course_id):
    """
    TODO Fix how this training data is sampled
    e.g. bootstrap sampling of a random number of courses
    to get a total of > 1 million training samples
    """

    # train = pd.read_csv('{}/{}/model_data.csv'.format(get_data_path(), 'Microsoft+DAT206x+3T2017'))
    train = None
    past_course_ids = [f for f in os.listdir(get_data_path()) if not f.startswith('.')]
    try:
        past_course_ids.remove(current_course_id)
    except ValueError:
        print('Not in list')

    for course_id in past_course_ids:
        if '4T2017' not in course_id:
            try:
                # course_run_data = pd.read_csv('{}/{}/model_data.csv'.format(get_data_path(), course_id))
                path = '{}/{}/model_data_l.csv'.format(get_data_path(), course_id)
                course_run_data = pd.read_csv(path)
            except Exception:
                print('model_data.csv does not exist for course: ', course_id)
                continue
                # pass                
            if train is None:
                train = course_run_data
            else:
                train = train.append(course_run_data)

    print('Training data done.')

    train = train.reset_index(drop=True)
    # test = pd.read_csv('{}/{}/model_data.csv'.format(get_data_path(), current_course_id))
    test = pd.read_csv('{}/{}/model_data_l.csv'.format(get_data_path(), current_course_id))

    X_cols = [
        'course_week', 'num_video_plays', 'num_problems_attempted',
        'num_problems_correct', 'num_subsections_viewed', 'num_forum_posts',
        'num_forum_votes', 'avg_forum_sentiment', 
        'user_started_week', 'user_active_previous_week'
    ]

    scaler = StandardScaler()
    scaler.fit(train[X_cols])

    X_train = scaler.transform(train[X_cols])
    X_test = scaler.transform(test[X_cols])

    X_train = np.array(X_train).astype(np.float32)
    X_test = np.array(X_test).astype(np.float32)

    y_train = np.array(train['user_dropped_out_next_week']).astype(np.float32)
    y_test = np.array(test['user_dropped_out_next_week']).astype(np.float32)

    return (X_train, y_train, X_test, y_test)


def _create_pivot_table(df, val_col):
    df_pivot = df.pivot_table(
        index='user_id', columns=['course_week'], values=val_col, fill_value=-1
    )
    df_colored = df_pivot.style.applymap(_cell_colors)
    return df_colored

def _cell_colors(s):
    ret = 'background-color: {}'
    if s == 0:
        ret = ret.format('#228b22')
    elif s == 1:
        ret = ret.format('#dc143c')
    else:
        ret = ret.format('#d3d3d3')

    return ret


# Function to create model, required for KerasClassifier
def create_model(model_input, hidden_layers_conf=[], name=''):
    # create model
    model = None
    
    for i, layer in enumerate(hidden_layers_conf):
        
        if i == 0:            
            model = Dense(layer['n_units'])(model_input)            
                
        model = BatchNormalization()(model)
        model = Activation('relu')(model)
        model = Dropout(layer.get('dropout', 0.2))(model)
    
    model = Dense(1)(model)
    model = BatchNormalization()(model)
    predictions = Activation('sigmoid')(model)
    
    model = Model(inputs=model_input, outputs=predictions)
    if name:
        model.name = name

    return model


def compile_and_train(model, x_train, y_train, optimizer='adam', metrics=['accuracy'], num_epochs=20, batch_size=100): 
    
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=metrics) 
    filepath = 'weights/' + model.name + '.{epoch:02d}-{loss:.2f}.hdf5'
    checkpoint = ModelCheckpoint(
        filepath, monitor='loss', verbose=0, save_weights_only=True, save_best_only=True, mode='auto', period=1
    )
    
    history = model.fit(x=x_train,
                        y=y_train, 
                        batch_size=batch_size, 
                        epochs=num_epochs, 
                        verbose=1,
                        class_weight={ 0: 1., 1: 2 },
                        callbacks=[checkpoint])

    return history


def ensemble_models(models, model_input):
    # collect outputs of models in a list
    print(models[0].outputs)
    model_outputs = [model.outputs[0] for model in models]
    
    print('OUTPUTS:')
    print(model_outputs)

    # averaging outputs
    avg = Average()(model_outputs)
    
    print('AVG:')
    print(avg)

    # build model from same input and avg output
    model_ens = Model(inputs=model_input, outputs=avg, name='ensemble') 
    print(model_ens)
   
    return model_ens


def fit_score_predict(course_id, train=False):

    print('GETTING DATA: ')
    X_train, y_train, X_test, y_test = get_data(course_id)
    print('Done.')

    input_shape = (X_train.shape[1],)
    model_input = Input(shape=input_shape)
    
    models = []
    batch_size = 256
    
    if not train:
        current_date_string = datetime.strftime(datetime.today(), '%Y-%m-%d')
        model = load_model('model.h5-{}'.format('2018-01-23'))
    else:
        adam = optimizers.Adam(lr=0.01)
        layers_conf = [
            {
                'n_units': 8,
                'dropout': 0.2,
                'input_dim': 10
            },
            {
                'n_units': 8,
                'dropout': 0.25
            },
            {
                'n_units': 6,
                'dropout': 0.2
            },
            {
                'n_units': 2,
                'dropout': 0.
            }
        ]

        print('Fitting model')
        
        kfold = StratifiedKFold(n_splits=3, shuffle=True)
        
        
        for i, (train_ind, val_ind) in enumerate(kfold.split(X_train, y_train)):
            
            model = create_model(model_input, 
                                 hidden_layers_conf=layers_conf, 
                                 name='kfold-{}'.format(i))
            
            _ = compile_and_train(model,
                                  num_epochs=3,
                                  batch_size=batch_size,
                                  train_data=(X_train[train_ind], y_train[train_ind]), 
                                  validation_data=(X_train[val_ind], y_train[val_ind]))            
            
            scores = model.evaluate(X_train[val_ind], y_train[val_ind], batch_size)
            
            models.append(model)
        
    print('Evaluating model on data for course: {}'.format(course_id))
    
    print(model_input)
    ensemble = ensemble_models(models, model_input)

    print('Done')

    try:
        current_date_string = datetime.strftime(datetime.today(), '%Y-%m-%d')
        ensemble.save('model-ensemble-{}.h5'.format(current_date_string))
    except:
        print('FAILED TO SAVE MODEL')
    
#     score = ensemble.evaluate(X_test, y_test, batch_size)
#     print('Model score', score)

    preds = ensemble.predict(X_test, batch_size)
    final_preds = np.round(preds)

    print('PREDS: ', final_preds)
    print('Y_TEST: ', y_test)

    conf_matrix = metrics.confusion_matrix(y_test, final_preds)

    tn, fp, fn, tp = conf_matrix.ravel()
    total = len(y_test)
    final_acc = (tn + tp) / total

    test_data_orig = pd.read_csv('{}/{}/model_data_l.csv'.format(get_data_path(), course_id))
    test_data_orig['predicted_user_dropped_out_next_week'] = final_preds
    

    pred_pivot = _create_pivot_table(test_data_orig, 'predicted_user_dropped_out_next_week')
    real_pivot = _create_pivot_table(test_data_orig, 'user_dropped_out_next_week')

    # save_df_to_file(pred_pivot, 'predicted_dropouts', course_id, type='excel')
    # save_df_to_file(real_pivot, 'real_dropouts', course_id, type='excel')
    # save_df_to_file(test_data_orig, 'model_data_with_preds', course_id)

#     save_df_to_file(pred_pivot, 'predicted_dropouts_l', course_id, type='excel')
#     save_df_to_file(real_pivot, 'real_dropouts_l', course_id, type='excel')
    save_df_to_file(test_data_orig, 'model_data_with_preds_l', course_id)

    print('ACCURACY: ', final_acc)
    print('CONFUSION MATRIX: ')
    print(conf_matrix)
    print(conf_matrix / len(y_test))

    return (final_preds, final_acc, conf_matrix)


In [38]:
fit_score_predict('Microsoft+DAT206x+4T2017', train=True)

GETTING DATA: 
model_data.csv does not exist for course:  Microsoft+DAT215.4x+1T2017
model_data.csv does not exist for course:  _SUCCESS
model_data.csv does not exist for course:  Microsoft+DAT206x+JPN+1T2017
model_data.csv does not exist for course:  Microsoft+DAT206x+6T2016
model_data.csv does not exist for course:  Microsoft+DAT206x+1T2018
model_data.csv does not exist for course:  Microsoft+DAT215.3x+3T2017
model_data.csv does not exist for course:  Microsoft+DAT206x+JPN+2T2017
model_data.csv does not exist for course:  Microsoft+DAT207x+1T2018
model_data.csv does not exist for course:  weights
model_data.csv does not exist for course:  top_course_ids.txt
model_data.csv does not exist for course:  Microsoft+DAT205x+3T2016
Training data done.
Done.
(10,)
Fitting model


/home/kabirkhan/Documents/ML_Experiments/py35/lib/python3.5/site-packages/keras/backend/cntk_backend.py:2337: UserWarning: CNTK backend warning: CNTK version not detected. Will using CNTK 2.0 GA as default.
  'CNTK backend warning: CNTK version not detected. '


Epoch 1/1
    800/1317788 [..............................] - ETA: 5:29 - loss: 1.1895 - acc: 0.5062 

/home/kabirkhan/Documents/ML_Experiments/py35/lib/python3.5/site-packages/cntk/core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input75544") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))


658894/658894 [==============================] - 5s 8us/step
acc: 75.15%
Epoch 1/1
    800/1317788 [..............................] - ETA: 5:41 - loss: 1.3244 - acc: 0.5400 

/home/kabirkhan/Documents/ML_Experiments/py35/lib/python3.5/site-packages/cntk/core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input78354") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))


658894/658894 [==============================] - 5s 8us/step
acc: 73.85%
Epoch 1/1
    700/1317788 [..............................] - ETA: 6:47 - loss: 1.2663 - acc: 0.4243 

/home/kabirkhan/Documents/ML_Experiments/py35/lib/python3.5/site-packages/cntk/core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input81164") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))


658894/658894 [==============================] - 5s 8us/step
acc: 77.02%
AVERAGE ACCURACY MANUAL: 75.34% (+/- 1.30%)
Evaluating model on data for course: Microsoft+DAT206x+4T2017
Input('input_18', [#], [10])
[Composite(StableSigmoid): Input('input_18', [#], [10]) -> Output('StableSigmoid75535_Output_0', [#], [1])]
OUTPUTS:
[Composite(StableSigmoid): Input('input_18', [#], [10]) -> Output('StableSigmoid75535_Output_0', [#], [1]), Composite(StableSigmoid): Input('input_18', [#], [10]) -> Output('StableSigmoid78345_Output_0', [#], [1]), Composite(StableSigmoid): Input('input_18', [#], [10]) -> Output('StableSigmoid81155_Output_0', [#], [1])]
AVG:
Composite(input_18: Tensor[10]) -> Tensor[1]
Done
PREDS:  [[ 1.]
 [ 0.]
 [ 1.]
 ..., 
 [ 1.]
 [ 1.]
 [ 1.]]
Y_TEST:  [ 0.  0.  1. ...,  0.  1.  1.]
ACCURACY:  0.740426738133
CONFUSION MATRIX: 
[[ 9330  7744]
 [ 2329 19403]]
[[ 0.24042674  0.19955677]
 [ 0.06001649  0.5       ]]


(array([[ 1.],
        [ 0.],
        [ 1.],
        ..., 
        [ 1.],
        [ 1.],
        [ 1.]], dtype=float32), 0.74042673813327831, array([[ 9330,  7744],
        [ 2329, 19403]]))

In [37]:
models = []
ensemble = None

In [68]:
y_true = np.array([1,1,1,0,0,1,1,0,0,1,1,0,0,1,1,0,0,1,1,0,1,1,0,0,1,1,1,0,1,0])
y_pred = np.array([1,1,1,0,0,1,1,0,0,1,1,0,0,1,1,1,1,1,0,0,1,1,0,0,1,1,0,0,1,1])

In [69]:
from sklearn import metrics

In [70]:
metrics.accuracy_score(y_true, y_pred)

0.83333333333333337

In [72]:
tn, fp, fn, tp = metrics.confusion_matrix(y_true, y_pred).ravel()

In [75]:
sensitivity = tp / float(fn + tp)
sensitivity

0.88235294117647056

In [76]:
metrics.recall_score(y_true, y_pred)

0.88235294117647056

In [78]:
fn / float(tp + fn)

0.11764705882352941

In [80]:
metrics.roc_auc_score(y_true, y_pred)

0.82579185520361975

In [82]:
metrics.recall_score(y_true, y_pred) - metrics.accuracy_score(y_true, y_pred)

0.049019607843137192

-inf

In [87]:
h = data.head(100)

In [89]:
h.describe()

,user_id,course_week,num_video_plays,num_problems_attempted,num_problems_correct,num_subsections_viewed,num_forum_posts,num_forum_votes,avg_forum_sentiment,user_started_week,user_last_active_week,user_completed_week,user_active_previous_week,user_dropped_out_next_week,predicted_user_dropped_out_next_week
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.0,100.0,100.0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,54812.970000,7.000000,8.700000,3.960000,2.670000,5.460000,0.0,0.0,0.0,5.210000,9.020000,-0.780000,0.240000,0.650000,0.770000
std,27943.162048,4.211576,19.145113,11.704199,8.520984,8.832241,0.0,0.0,0.0,2.962032,5.235813,1.547758,0.429235,0.479372,0.422953
min,1131.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,1.000000,1.000000,-1.000000,0.000000,0.000000,0.000000
25%,33522.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,3.000000,4.000000,-1.000000,0.000000,0.000000,1.000000
50%,58444.000000,7.000000,1.000000,0.000000,0.000000,4.000000,0.0,0.0,0.0,6.000000,8.000000,-1.000000,0.000000,1.000000,1.000000
75%,82020.000000,10.000000,8.500000,2.250000,2.000000,7.000000,0.0,0.0,0.0,7.000000,15.000000,-1.000000,0.000000,1.000000,1.000000
max,84693.000000,16.000000,111.000000,101.000000,76.000000,68.000000,0.0,0.0,0.0,13.000000,16.000000,10.000000,1.000000,1.000000,1.000000


In [108]:
h[['num_video_plays', 'num_problems_attempted',
       'num_problems_correct', 'num_subsections_viewed', 'num_forum_posts',
       'num_forum_votes', 'avg_forum_sentiment']].describe()

,num_video_plays,num_problems_attempted,num_problems_correct,num_subsections_viewed,num_forum_posts,num_forum_votes,avg_forum_sentiment
count,100.000000,100.000000,100.000000,100.000000,100.0,100.0,100.0
mean,8.700000,3.960000,2.670000,5.460000,0.0,0.0,0.0
std,19.145113,11.704199,8.520984,8.832241,0.0,0.0,0.0
min,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
25%,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0
50%,1.000000,0.000000,0.000000,4.000000,0.0,0.0,0.0
75%,8.500000,2.250000,2.000000,7.000000,0.0,0.0,0.0
max,111.000000,101.000000,76.000000,68.000000,0.0,0.0,0.0


In [91]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.show()

In [ ]:
optimizers.Adam(lr=0.01, decay=1e-5)

In [110]:
0.01 / 10

0.001

In [131]:
lr = 0.1
decay_rate = 0.05
num_epochs = 10
for i in range(num_epochs):
    lr *= (1. / (1. + decay_rate * num_epochs))
    print(lr)

0.06666666666666667
0.04444444444444444
0.029629629629629624
0.019753086419753083
0.013168724279835387
0.008779149519890258
0.0058527663465935045
0.003901844231062336
0.0026012294873748905
0.0017341529915832603


In [120]:
from __future__ import print_function

from datetime import datetime
import sys

from azure.storage.file import FileService
import azure.mgmt.batchai.models as models

# utilities.py contains helper functions used by different notebooks
from batch_ai import utilities

cfg = utilities.Configuration('./batch_ai/configuration.json')
client = utilities.create_batchai_client(cfg)

In [126]:
model_path = 'https://learnerattrition.file.core.windows.net/batchaisample/fd74930d-c060-4ff4-a7f1-9470f7ad7f8f/learner-attrition-supp/jobs/la_01_30_2018_031346/4d6e71b4-e9d6-4b25-8aa0-3f7c136f09c7/outputs/Models/'
model_file_name = 'model.h5'
# utilities.download_file(model_file, model_name)

In [123]:
service = FileService(cfg.storage_account_name, cfg.storage_account_key)

In [ ]:
with open('./model-stream.h5', 'wb') as stream:
    service.get_file_to_stream('batchaisample', model_path, model_file_name, stream)

# LSTM Testing

In [135]:
data = pd.read_csv('./data/Microsoft+DAT206x+1T2017/model_data_l.csv')

In [137]:
data.head(10)

,user_id,course_week,num_video_plays,num_problems_attempted,num_problems_correct,num_subsections_viewed,num_forum_posts,num_forum_votes,avg_forum_sentiment,user_started_week,user_last_active_week,user_completed_week,user_active_previous_week,user_dropped_out_next_week
0,1102,11.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,11.0,16.0,-1.0,0.0,0.0
1,1102,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,16.0,-1.0,1.0,0.0
2,1102,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,16.0,-1.0,0.0,1.0
3,1102,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,16.0,-1.0,0.0,1.0
4,1102,15.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,16.0,-1.0,0.0,1.0
5,1102,16.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,11.0,16.0,-1.0,0.0,1.0
6,1547,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,7.0,7.0,-1.0,0.0,1.0
7,1547,7.0,0.0,1.0,1.0,10.0,0.0,0.0,0.0,7.0,7.0,-1.0,0.0,1.0
8,2067,9.0,26.0,12.0,6.0,45.0,0.0,0.0,0.0,9.0,9.0,-1.0,0.0,1.0
9,3393,1.0,91.0,37.0,32.0,22.0,0.0,0.0,0.0,1.0,14.0,6.0,0.0,0.0


In [138]:
data.describe()

,user_id,course_week,num_video_plays,num_problems_attempted,num_problems_correct,num_subsections_viewed,num_forum_posts,num_forum_votes,avg_forum_sentiment,user_started_week,user_last_active_week,user_completed_week,user_active_previous_week,user_dropped_out_next_week
count,1.595070e+05,159507.000000,159507.000000,159507.000000,159507.000000,159507.000000,159507.000000,159507.000000,159507.000000,159507.000000,159507.000000,159507.000000,159507.000000,159507.000000
mean,1.032366e+07,13.312989,5.239538,4.111932,2.987023,4.403681,0.000542,0.017429,0.000796,5.217489,21.691525,0.607321,0.165397,0.702665
std,3.931041e+06,10.633514,22.624566,14.612374,11.344113,10.030069,0.017953,0.654222,0.082396,3.418082,15.759889,4.342588,0.371540,0.457087
min,1.102000e+03,-1.000000,0.000000,0.000000,0.000000,0.000000,-0.800000,0.000000,0.000000,-1.000000,-1.000000,-1.000000,0.000000,0.000000
25%,8.060308e+06,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,9.000000,-1.000000,0.000000,0.000000
50%,1.227390e+07,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,15.000000,-1.000000,0.000000,1.000000
75%,1.325299e+07,17.000000,1.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,8.000000,35.000000,-1.000000,0.000000,1.000000
max,1.416629e+07,55.000000,1340.000000,203.000000,200.000000,241.000000,1.000000,137.000000,27.000000,13.000000,55.000000,13.000000,1.000000,1.000000


In [177]:
data = data.iloc[data.loc[data.apply(lambda x: np.abs(x - x.mean()) / x.std() < 5).all(axis=1), [
    'num_video_plays', 'num_problems_attempted',
    'num_problems_correct', 'num_subsections_viewed', 'num_forum_posts',
    'num_forum_votes', 'avg_forum_sentiment'
]].index]

In [178]:
data.describe()

,user_id,course_week,num_video_plays,num_problems_attempted,num_problems_correct,num_subsections_viewed,num_forum_posts,num_forum_votes,avg_forum_sentiment,user_started_week,user_last_active_week,user_completed_week,user_active_previous_week,user_dropped_out_next_week
count,1.560300e+05,156030.000000,156030.000000,156030.000000,156030.000000,156030.000000,156030.000000,156030.000000,156030.0,156030.000000,156030.000000,156030.000000,156030.000000,156030.000000
mean,1.031412e+07,13.438916,3.387208,2.594975,1.821284,3.452624,0.000013,0.002730,0.0,5.210831,21.884618,0.454913,0.159027,0.715670
std,3.934641e+06,10.689321,10.802378,8.372761,6.370200,6.594441,0.001624,0.064486,0.0,3.413809,15.808410,4.169342,0.365703,0.451096
min,1.102000e+03,-1.000000,0.000000,0.000000,0.000000,0.000000,-0.086111,0.000000,0.0,-1.000000,-1.000000,-1.000000,0.000000,0.000000
25%,8.047784e+06,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,2.000000,9.000000,-1.000000,0.000000,0.000000
50%,1.226006e+07,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,5.000000,16.000000,-1.000000,0.000000,1.000000
75%,1.325213e+07,18.000000,1.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.0,8.000000,35.000000,-1.000000,0.000000,1.000000
max,1.416629e+07,55.000000,118.000000,77.000000,59.000000,54.000000,0.089286,3.000000,0.0,13.000000,55.000000,13.000000,1.000000,1.000000


In [183]:
# function to reshape features into (samples, time steps, features) 
def gen_sequence(id_df, seq_length):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    data_array = id_df.values
    num_elements = data_array.shape[0]
    for start, stop in zip(range(0, num_elements-seq_length), range(seq_length, num_elements)):
        yield data_array[start:stop, :]

In [ ]:
def create_lstm():
    
    # build the network
    nb_features = seq_array.shape[2]
    nb_out = label_array.shape[1]

    model = Sequential()

    model.add(LSTM(
             input_shape=(sequence_length, nb_features),
             units=100,
             return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(
              units=50,
              return_sequences=False))
    model.add(Dropout(0.2))

    model.add(Dense(units=nb_out, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model


In [224]:
from sklearn.model_selection import StratifiedKFold
from keras.preprocessing import sequence

X_cols = [
    'user_id',
    'course_week', 'num_video_plays', 'num_problems_attempted',
    'num_problems_correct', 'num_subsections_viewed', 'num_forum_posts',
    'num_forum_votes', 'avg_forum_sentiment', 
    'user_started_week', 'user_active_previous_week'
]
X = data[X_cols]
y = data[['user_id', 'user_dropped_out_next_week']]



kfold = StratifiedKFold(n_splits=3, shuffle=True)
    
X_train, y_train, X_val, y_val = None, None, None, None
    
for i, (train_ind, val_ind) in enumerate(kfold.split(X, y['user_dropped_out_next_week'])):

    X_train, y_train = X.iloc[train_ind], y.iloc[train_ind]
    X_val, y_val = X.iloc[val_ind], y.iloc[val_ind]
    break
    
print('Sliced, now normalizing')

scaler = StandardScaler()
scaler.fit(X_train.drop('user_id', axis=1))

X_train_scaled = scaler.transform(X_train.drop('user_id', axis=1))
X_val_scaled = scaler.transform(X_val.drop('user_id', axis=1))

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_cols[1:])
X_train_scaled['user_id'] = X_train['user_id']
X_val_scaled = pd.DataFrame(X_val_scaled, columns=X_cols[1:])
X_val_scaled['user_id'] = X_val['user_id']

print('Scaled, now generating sequences')
sequence_length = 4
seq_gen = (list(gen_sequence(X_train_scaled[X_train_scaled['user_id']==id], sequence_length)) 
       for id in X_train_scaled['user_id'].unique())
print('Seq Gen')

# generate sequences and convert to numpy array
seq_array = np.concatenate(sequence.pad_sequences(list(seq_gen), maxlen=4)).astype(np.float32)
print(seq_array.shape)
print('Done')

print('Labels Gen')
# function to generate labels
def gen_labels(id_df, seq_length, label):
    data_array = id_df[label].values
    num_elements = data_array.shape[0]
    return data_array[seq_length:num_elements, :]

# generate labels
label_gen = [gen_labels(y_train[y_train['user_id']==id], sequence_length, ['user_dropped_out_next_week']) 
             for id in y_train['user_id'].unique()]

label_array = np.concatenate(sequence.pad_sequences(list(label_gen), maxlen=4)).astype(np.float32)
label_array.shape
print(label_array.shape)
print('Done')

#     X_train = np.array(X_train_scaled).astype(np.float32)
#     X_val = np.array(X_val_scaled).astype(np.float32)

#     y_train = np.array(y_train).astype(np.float32)
#     y_val = np.array(y_val).astype(np.float32)

#     model = create_model(model_input, 
#                          hidden_layers_conf=layers_conf, 
#                          name='kfold-{}'.format(i))



#     scores = model.evaluate(X_train[val_ind], y_train[val_ind], batch_size)

#     models.append(model)

Sliced, now normalizing
Scaled, now generating sequences
Seq Gen
(69032, 4, 11)
Done
Labels Gen
(114868, 1)
Done


In [238]:
for id in X_train_scaled['user_id'].unique():
    print(id)
    break

nan


In [202]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(104020, 11)
(104020, 2)
(52010, 11)
(52010, 2)


In [223]:
pd.DataFrame(label_array)[0].value_counts()

0.0    1079419
1.0      40388
Name: 0, dtype: int64

[[0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],
 [0.0],


In [179]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

data_dim = 16
timesteps = 8
num_classes = 10

# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(LSTM(32, return_sequences=True,
               input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32, return_sequences=True))  # returns a sequence of vectors of dimension 32
model.add(LSTM(32))  # return a single vector of dimension 32
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Generate dummy training data
x_train = np.random.random((1000, timesteps, data_dim))
y_train = np.random.random((1000, num_classes))

# Generate dummy validation data
x_val = np.random.random((100, timesteps, data_dim))
y_val = np.random.random((100, num_classes))

model.fit(x_train, y_train,
          batch_size=64, epochs=5,
          validation_data=(x_val, y_val))

/home/kabirkhan/Documents/ML_Experiments/py35/lib/python3.5/site-packages/keras/backend/cntk_backend.py:2337: UserWarning: CNTK backend warning: CNTK version not detected. Will using CNTK 2.0 GA as default.
  'CNTK backend warning: CNTK version not detected. '
/home/kabirkhan/Documents/ML_Experiments/py35/lib/python3.5/site-packages/cntk/core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input85523") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))
/home/kabirkhan/Documents/ML_Experiments/py35/lib/python3.5/site-packages/cntk/core.py:361: UserWarning: your data is of type "float64", but your input variable (uid "Input83776") expects "<class 'numpy.float32'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))


Train on 1000 samples, validate on 100 samples
Epoch 1/5
1000/1000 [==============================] - 0s 472us/step - loss: 11.5202 - acc: 0.0850 - val_loss: 11.6669 - val_acc: 0.0800
Epoch 2/5
1000/1000 [==============================] - 0s 350us/step - loss: 11.5180 - acc: 0.1000 - val_loss: 11.6697 - val_acc: 0.0800
Epoch 3/5
1000/1000 [==============================] - 0s 353us/step - loss: 11.5177 - acc: 0.1100 - val_loss: 11.6661 - val_acc: 0.1500
Epoch 4/5
1000/1000 [==============================] - 0s 360us/step - loss: 11.5175 - acc: 0.1030 - val_loss: 11.6669 - val_acc: 0.1400
Epoch 5/5
1000/1000 [==============================] - 0s 340us/step - loss: 11.5173 - acc: 0.0920 - val_loss: 11.6662 - val_acc: 0.1000


In [181]:
x_train.shape

(1000, 8, 16)

In [240]:
a = np.array([0,1,0,1,1])

In [241]:
from collections import Counter

In [244]:
Counter(a)

Counter({0: 2, 1: 3})

In [246]:
accuracy_score(a, np.ones(a.shape))

0.59999999999999998

In [248]:
p = np.array([0.84, 0.22, 0.9, 0.5])

0.61499999999999999

In [260]:
def create_model(model_input, hidden_layers_conf=[], name=''):
    # create model
    model = None
    
    for i, layer in enumerate(hidden_layers_conf):
        
        if i == 0:            
            model = Dense(layer['n_units'], name='Fully_Connected_Input')(model_input)
        else:
            model = Dense(layer['n_units'], name='Fully_Connected_{}'.format(i + 1))(model)

        model = BatchNormalization(name='Batch_Normalize_{}'.format(i + 1))(model)
        model = Activation('relu', name='ReLU_{}'.format(i + 1))(model)
        model = Dropout(layer.get('dropout', 0.3), name='Dropout_{}'.format(i + 1))(model)
    
    model = Dense(1, name='Output')(model)
    model = BatchNormalization(name='Batch_Normalize_Output')(model)
    predictions = Activation('sigmoid', name='Sigmoid')(model)
    
    model = Model(inputs=model_input, outputs=predictions)
    if name:
        model.name = name

    return model

with open('./batch_ai/params.json', 'r') as f:
    import json
    layers_conf = json.loads(f.read())["layers"]

print('Fitting model')
# from keras.layers import 
model_input = Input(shape=(10,), name='Input')

model = create_model(model_input, 
                                 hidden_layers_conf=layers_conf, 
                                 name='kfold')

Fitting model


/home/kabirkhan/Documents/ML_Experiments/py35/lib/python3.5/site-packages/keras/backend/cntk_backend.py:2337: UserWarning: CNTK backend warning: CNTK version not detected. Will using CNTK 2.0 GA as default.
  'CNTK backend warning: CNTK version not detected. '


In [261]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (None, 10)                0         
_________________________________________________________________
Fully_Connected_Input (Dense (None, 8)                 88        
_________________________________________________________________
Batch_Normalize_1 (BatchNorm (None, 8)                 32        
_________________________________________________________________
ReLU_1 (Activation)          (None, 8)                 0         
_________________________________________________________________
Dropout_1 (Dropout)          (None, 8)                 0         
_________________________________________________________________
Fully_Connected_2 (Dense)    (None, 8)                 72        
_________________________________________________________________
Batch_Normalize_2 (BatchNorm (None, 8)                 32        
__________